## Live Well helper functions
Functions developed to specifically assist processing of Live Well Dorset wellbeing service data ready for further analysis

In [ ]:
# Load relevant libraries 
import pandas as pd
import numpy as np
import math

### 1. Generate categorical values from client (demographic) data set
This function extrapolates categorical values for use during stratification and analysis. Results are returned as additional fields in the supplied pandas data frame. 
* Gender (binary numeric) 
* Month of year
* Age groups (10 year bins)
* Binary numeric GP referal source 


In [ ]:
def prepare_clients(df):
    # Numericise the gender field for arithmetic corrlations
    df['gender_numeric'] = np.nan
    df['gender_numeric'][df.Gender == 'Male'] = 0
    df['gender_numeric'][df.Gender == 'Female'] = 1

    # Bin dates by month
    df['YearMonth'] = df.Date_registered_Month_Year.map(lambda x: x.strftime('%Y-%m'))

    # Bin ages by 10 year divisions
    groups = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']
    df['age_group'] = pd.cut(df.Age, range(0, 101, 10), right=False, labels=groups)

    # Mark GP referals by practice staff and those with "Doctor" in the referal source identifier that are not hospital related
    ref_filter = ['GP','Doctor','Doctor, Community Group','Doctor, Doctors','Practice Nurse','Health Care Assistant, Doctors','Midwife, Doctors','Nurse, Doctors','Practice Nurse, Doctors']
    df['gp_referral'] = 0
    df['gp_referral'].loc[df['ReferralSource'].isin(ref_filter) | df['how_a'].isin(ref_filter) | df['how_a'].isin(ref_filter) | df['referral_combined'].isin(ref_filter)] = 1
    df['gp_referral'].sum()

    # Mark GP referals by practice staff and those with "Doctor" in the referal source identifier that are not hospital related
    ref_filter = ['Doctor','Doctor, Community Group','Doctor, Doctors','Practice Nurse','Health Care Assistant, Doctors','Midwife, Doctors','Nurse, Doctors','Practice Nurse, Doctors']
    df['gp_referral'] = 0
    df['gp_referral'].loc[df['how_a'].isin(ref_filter) | df['referral_combined'].isin(ref_filter)] = 1

    # Dataframe of unkown referal sources
    #ref_filter = ['Not asked','Other']
    #unkown_refs = df.loc[df['how_a'].isin(ref_filter) & df['referral_combined'].isin(ref_filter)]

    return df

### 2. Output a list if all referral types
This function returns a list of the unqiue referral sources found within the data set

In [ ]:
def referral_types(df, fields=['how_a','referral_combined']):
    return pd.unique(df[fields].values.ravel('K'))

### 3. Count the number of unique referal types
This function includes the number of records found for each of the unique referral source classificaitons

In [20]:
def get_unique_referal_types(df):
    return pd.unique(df[['ReferralSource','how_a','referral_combined']].values.ravel('K'))

### 4. Smoking quantity standardisation 
This function compares multiple related columns of varying data types and decides to what level the client is/was a smoker and trasnlates this to a simplified numerical categorisation.
* 0 - Non smoker
* 1 - Smokes 1 to 9 times a day
* 2 - Smokes 10 to 19 times a day
* 3 - Smokes 20 or more times a day

In [ ]:
def smoking_class(values):
    # Define standardise codes for various data values
    mapping = { 'a. 1-9': 1, 'a.': 1, 'b. 10-19': 2, 'c. 20+': 3, 'New non smoker': 0, 'Non-Smoker': 0, 'Non smoker': 0, 'Cigarettes': 1, 'e-Cigarettes': 1, 'Roll-ups': 1, 'Smoking (cigarettes, cigars, pipe, roll-ups)': 1, 'E-cigs / vape (E-cigarettes)': 1 }
    missing_flag = False
    zero_flag = False
    coded = False
    for value in values:
        if value in ['Missing']:
            missing_flag = True
        if value in ['0',0]:
            zero_flag = True
        if value in mapping:
            coded = mapping[value]
            continue
        if not coded:
            if value.isdigit():
                value = pd.to_numeric(value)
                if value >= 1 and value <= 9:
                    coded = 1
                elif value < 19:
                    coded = 2
                elif value >= 20:
                    coded = 3
    if not coded or missing_flag and zero_flag:
        return np.nan
    else:
        return coded

### 6. Stadardise case pathway data
This function reviews and consolidates tracking data for the 4 service pathways offered by Live Well Dorset. Through initial analysis is was found that numerous aspects of the data were incomplete or poorly structures (multiple interpretable values). The formalised variables are appended to the original data frame for use in further analysis. Definitions of each pathway 'success' state are detailed in the accompanying thesis document.

In [10]:
def prepare_cases(df):
    # Loop all case records
    for index, row in df.iterrows():
        pathway_count = 0
        followup_count = []
        pathways_success = np.NaN
         # Smoking reduction pathway
        if row['Smoking_PathwayActivatedFlag'] == 1:
            pathway_count += 1
            # Ignore rows who start as non smokers (this should be captured with the above, but better to be sure)
            if (row['Smok_Init_Qty_Day_Group'] != '0') & (row['Smoking_InitialQtyPerDay'] != 'Non smoker') & (row['Smoking_InitialQtyPerDay'] != '0'):
                smoking_followups = 0
                smoking_success = 0
                # Standardise the smoking quantities
                smoking_df = {}
                start_smoking = smoking_class(row[['Smoking_InitialQtyPerDay','Smok_Init_Qty_Day_Group']].values)
                smoking_df['Smoker3'] = smoking_class(row[['Smoking_ThreeMthFUpQtyPerDay','Smoking_3_Month_Daily_Quantity']].values)
                smoking_df['Smoker6'] = smoking_class(row[['Smoking_SixMthFUpQtyPerDay','Smoking_6_Month_Daily_Quantity']].values)
                smoking_df['Smoker12'] = smoking_class(row[['Smoking_TwelveMthFUpQtyPerDay','Smok_12_Months_Qty_Day_group']].values)
                # Process each potential followup
                for key in smoking_df:
                    if not math.isnan(smoking_df[key]):
                        smoking_followups += 1
                        end_smoking = smoking_df[key]
                # Flag if smoking has reduced
                net_smoking_reduction = float('nan')
                if end_smoking < start_smoking:
                    smoking_success = 1
                    pathways_success = 1
                    smoking_net_reduction = start_smoking - end_smoking
                else:
                    if smoking_followups > 0:
                        smoking_success = 0
                    else:
                        smoking_success = -1
                df.at[index,'smoking_followups'] = smoking_followups
                followup_count.append(smoking_followups)
                df.at[index,'smoking_success'] = smoking_success

        # Weight loss pathway
        if row['Weight_PathwayActivatedFlag'] == 1 and row['Weight_Initial'] > 0:
            pathway_count += 1
            weight_followups = 0
            weight_success = 0
            end_weight = row['Weight_Initial']
            for field in ['Weight_ThreeMthFUpWeight','Weight_SixMthFUpWeight','Weight_TwelveMthFUpWeight']:
                # Process each potential followup
                if row[field] > 0:
                    weight_followups += 1
                    end_weight = row[field]
            # Flag if weight has been lost
            net_weight_loss = float('nan')
            if end_weight < row['Weight_Initial']:
                weight_net_loss = row['Weight_Initial'] - end_weight
                weight_net_loss_percent = (row['Weight_Initial'] - end_weight) * (100/row['Weight_Initial'])
                if weight_net_loss_percent > 5: # Success is a 5% loss or more
                    weight_success = 1
                    pathways_success = 1
                else:
                    weight_success = 0
            else:
                if weight_followups > 0:
                    weight_success = 0
                else:
                    weight_success = -1
            df.at[index,'weight_followups'] = weight_followups
            followup_count.append(weight_followups)
            df.at[index,'weight_success'] = weight_success
            df.at[index,'net_weight_loss'] = net_weight_loss

        # Alcohol reduction pathway
        # Ignore rows who start as non drinkers (this should be captured with the above, but better to be sure)
        if row['Alcohol_PathwayActivatedFlag'] == 1 and row['Alcohol_InitialAlcoholUnit'] != 0:
            pathway_count += 1
            alcohol_followups = 0
            alcohol_success = 0
            end_alcohol = row['Alcohol_InitialAlcoholUnit']
            for field in ['Alcohol_ThreeMthFUpAlcoholUnit','Alcohol_SixMthFUpAlcoholUnit','Alcohol_TwelveMthFUpAlcoholUnit']:
                # Process each potential followup
                if row[field] != -1:
                    alcohol_followups += 1
                    end_alcohol = row[field]
            # Flag is alcohol has reduced
            net_alcohol_reduction = float('nan')
            if end_alcohol < row['Alcohol_InitialAlcoholUnit']: # This is not units, they have to have droppped a banding to be 'successful'
                alcohol_success = 1
                pathways_success = 1
            else:
                pathways_success = 0
                # alcohol_net_reduction = row['Alcohol_InitialAlcoholUnit'] - end_alcohol This is a categorised value - not calculable
            df.at[index,'alcohol_followups'] = alcohol_followups
            followup_count.append(alcohol_followups)
            if alcohol_followups > 0:
                df.at[index,'alcohol_success'] = alcohol_success
            else:
                # If there were no followups it is unfair to say the pathway did not succeed
                df.at[index,'alcohol_success'] = -1
            df.at[index,'net_alcohol_reduction'] = net_alcohol_reduction

        # Increased activity pathway
        if row['Activity_PathwayActivatedFlag'] == 1 and row['Activity_InitialActivityLevel'] > -1:
            pathway_count += 1
            activity_followups = 0
            activity_success = 0
            end_activity = row['Activity_InitialActivityLevel']
            for field in ['Activity_ThreeMthFUpActivityLevel','Activity_SixMthFUpActivityLevel','Activity_TwelveMthFUpActivityLevel']:
                # Process each potential followup
                if row[field] != -1:
                    activity_followups += 1
                    end_activity = row[field]
            # Flag is activity has increased
            net_activity_increase = float('nan')
            if end_activity > row['Activity_InitialActivityLevel']: # This is not units, they have to have droppped a banding to be 'successful'
                activity_success = 1
                pathways_success = 1
            else:
                pathways_success = 0
                # alcohol_net_reduction = row['Alcohol_InitialAlcoholUnit'] - end_alcohol This is a categorised value - not calculable
            df.at[index,'activity_followups'] = activity_followups
            followup_count.append(activity_followups)
            if activity_followups > 0:
                df.at[index,'activity_success'] = activity_success
            else:
                # If there were no followups it is unfair to say the pathway did not succeed
                df.at[index,'activity_success'] = -1
            df.at[index,'net_activity_increate'] = net_activity_increase

        df.at[index,'pathways_success'] = pathways_success
        df.at[index,'pathway_count'] = pathway_count
        if len(followup_count):
            df.at[index,'followup_count'] = max(followup_count)
        else:
            followup_count = 0
        return df

### 6. Client-Case join
This function joins client records to the related cases.

In [11]:
def join_clients(df, clients):
    # Merge clients with cases data set
    df = pd.merge(df, clients, left_on='clientID', right_index=True)
    # Remove duplicate case IDs
    df.drop_duplicates(subset="CaseID", keep='first', inplace=True)
    return df

### 7. Format time periods for Causal Impact analysis
With a given daily time series dataframe and a specified intervention date, this function returns a pre and post from/to value pair for use with with the caulsal impact library. Intervals (in days) can be optionally include to override the defaults.

In [ ]:
def get_periods(df, intervention, post_interval = 28, pre_multiplier = 3):
    post_from = min(item for item in df.index if item > intervention)
    post_to = intervention + timedelta(days=post_interval)
    try:
        post_to = min(item for item in df.index if item > post_to)
    except ValueError:
        # We are past the end of the dataset, use the max and inform intead
        post_to = max(df.index)
        print('Warning; max series date used for post_to: ' + str(post_to) + '. Post period = ' + str(post_to - post_from))
    pre_from = intervention - timedelta(days=(post_interval * pre_multiplier))
    pre_from = min(item for item in df.index if item > pre_from)
    # Get date immediatly before intervention
    loc = df.index.get_loc(intervention)
    pre_to = df.index[loc]
    return [str(pre_from.date()),str(pre_to.date())], [str(post_from.date()),str(post_to.date())];

### 8. Structure data for Facebook Prophet library
Manipulate dataframe time series to correct structure and naming for use with Facebook Prophet library

In [ ]:
def prepare_for_prophet(df, gpref=None, start=False,end=False):
    if gpref is not None:
        # Filter by GP referals
        if gpref:
            df = df[(df.gp_referral == 1)]
        else:
            df = df[(df.gp_referral != 1)]

    # Drop dates before start date
    if start:
        df = df[(df.DateRegistered >= start)]

    # Drop dates after end date
    if end:
        df = df[(df.DateRegistered <= end)]

    # Reduce to single date field
    df = df[['DateRegistered']]
    # Count daily registrations
    df = df.groupby([df["DateRegistered"].dt.year, df["DateRegistered"].dt.month, df["DateRegistered"].dt.day]).count()


    # Convert index to datetime data type
    a = pd.DataFrame(df.index.values.tolist(), columns=['year','month','day'])
    df.index = pd.to_datetime(a)

    # Format dataframe for FB Prophet library
    df = df.reset_index()
    df.columns = ['ds', 'y']
    return df

### 9. Generate table data outside of Jupyter notebook  
This function is to assist with outputting tabular data for use in other formats

In [15]:
def output_html_table_file(mytable):
    data, metadata = get_ipython().display_formatter.format(mytable)
    with open('tables/referal-demographic-stratification.html', 'w') as f:
        f.write(data['text/html'])  # Assuming the object has an HTML representation